# Wir implementieren einen Hund-Katze Klassifikator mit CNNs

In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
import tensorflow as tf


from tensorflow.keras import utils
from imutils import paths
import numpy as np
import argparse
import cv2
import os

## Folgende Funktion nimmt ein Bild entgegen und skaliert das Bild auf 32x32 RGB-Pixel

In [2]:
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities

    return cv2.resize(image, size)
#cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))

## Laden Sie die Daten aus der Übung zu Neuronalen Netzen (Hund-Katze-Klassifikator):

Schreiben Sie eine Funktion, die die Bilder einliest, mit Hilfe der Funktion image_to_feature_vector vorverarbeiten. Anschliessend sollen die Vektoren in eine data-Matrix geschrieben werden, 
und ein label-Vektor ist zu erzeugen. 



In [3]:
def getTestData(path):
    data = os.listdir(path)
    Xtrain = np.zeros((len(data),32,32,3)) # changed here to fit 32*32 RGB Pixels instead of 3072
    ytrain = np.zeros((len(data)))
    i=0
    for j in range(len(data)):
        imgName=data[j]
        i=i+1
        if((len(data)-i)%1000 ==0):
            print(len(data)-i)
        img = cv2.imread(os.path.join(path,imgName))
        Xtrain[j]=image_to_feature_vector(img)
        if("cat" in imgName):
            ytrain[j]=0 #0=Cat
        else:
            ytrain[j]=1 #1=Dog
    print("Done")
    return Xtrain, ytrain

Xtrain, ytrain = getTestData("C:\\Users\\ABleicher\\Desktop\\train")

24000
23000
22000
21000
20000
19000
18000
17000
16000
15000
14000
13000
12000
11000
10000
9000
8000
7000
6000
5000
4000
3000
2000
1000
0
Done


## Splitten Sie die Daten in ein train/test-set.

Macht es Sinn die Daten zu normieren?

In [4]:
from sklearn.model_selection import train_test_split
Xtr=Xtrain #save
print(Xtrain.shape)
Xtrain /= 255
print(Xtrain.shape)

(Xtr, Xte, ytr, yte) = train_test_split(Xtrain, ytrain)

(25000, 32, 32, 3)
(25000, 32, 32, 3)


# Erstellen Sie ein CNN mit Keras

In [18]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(32,32,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ... und starten Sie das Training

In [20]:
print(ytr.shape)
model.fit(Xtr, ytr, epochs=15)

(18750,)
Epoch 1/15
586/586 [==============================] - 15s 25ms/step - loss: 0.4638 - accuracy: 0.7678
Epoch 2/15
586/586 [==============================] - 15s 26ms/step - loss: 0.4510 - accuracy: 0.7687
Epoch 3/15
586/586 [==============================] - 15s 25ms/step - loss: 0.4384 - accuracy: 0.7792
Epoch 4/15
586/586 [==============================] - 15s 25ms/step - loss: 0.4271 - accuracy: 0.7846
Epoch 5/15
586/586 [==============================] - 15s 25ms/step - loss: 0.4152 - accuracy: 0.7890
Epoch 6/15
586/586 [==============================] - 15s 25ms/step - loss: 0.4001 - accuracy: 0.7959
Epoch 7/15
586/586 [==============================] - 15s 25ms/step - loss: 0.3957 - accuracy: 0.7977
Epoch 8/15
586/586 [==============================] - 15s 25ms/step - loss: 0.3824 - accuracy: 0.8067
Epoch 9/15
586/586 [==============================] - 15s 25ms/step - loss: 0.3678 - accuracy: 0.8133
Epoch 10/15
586/586 [==============================] - 15s 25ms/step - lo

# Evaluieren Sie das Netz auf den held-back Test-Daten


Berechnen Sie die Akkuratheit und die Verwechselungsmatrix

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
predictTest = model.predict(Xte)
yPred = np.empty([0])
for y in predictTest:
    yPred= np.append(yPred,np.round(y))
print(confusion_matrix(yte, yPred))
print(accuracy_score(yte, yPred))

196/196 [==============================] - 1s 5ms/step
[[2284  831]
 [ 664 2471]]
0.7608


# Spielwiese

Vergleichen Sie: 

- unterschiedliche Netztopologien
- unterschiedliche Trainingsverfahren
- Dropout

In [36]:
def fitAndTestModel(modelDeclared):
    modelDeclared.fit(Xtr, ytr, epochs=15)
    predictTest = model.predict(Xte)
    yPred = np.empty([0])
    for y in predictTest:
        yPred= np.append(yPred,np.round(y))
    print(confusion_matrix(yte, yPred))
    print(accuracy_score(yte, yPred))

In [42]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(32,32,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
              
fitAndTestModel(model)

Epoch 1/15
586/586 [==============================] - 15s 24ms/step - loss: 0.6884 - accuracy: 0.5329
Epoch 2/15
586/586 [==============================] - 16s 27ms/step - loss: 0.6579 - accuracy: 0.6148
Epoch 3/15
586/586 [==============================] - 18s 30ms/step - loss: 0.6038 - accuracy: 0.6829
Epoch 4/15
586/586 [==============================] - 16s 28ms/step - loss: 0.5390 - accuracy: 0.7383
Epoch 5/15
586/586 [==============================] - 15s 25ms/step - loss: 0.4917 - accuracy: 0.7681
Epoch 6/15
586/586 [==============================] - 15s 26ms/step - loss: 0.4625 - accuracy: 0.7871
Epoch 7/15
586/586 [==============================] - 15s 26ms/step - loss: 0.4323 - accuracy: 0.8058
Epoch 8/15
586/586 [==============================] - 16s 27ms/step - loss: 0.4118 - accuracy: 0.8177
Epoch 9/15
586/586 [==============================] - 15s 26ms/step - loss: 0.3858 - accuracy: 0.8305
Epoch 10/15
586/586 [==============================] - 15s 25ms/step - loss: 0.363

In [45]:
from tensorflow.keras.optimizers import SGD
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(32,32,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(32))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

opt=SGD(lr=0.01, momentum=0.8)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
fitAndTestModel(model)

Epoch 1/15
586/586 [==============================] - 17s 28ms/step - loss: 0.6910 - accuracy: 0.5268
Epoch 2/15
586/586 [==============================] - 16s 27ms/step - loss: 0.6793 - accuracy: 0.5723
Epoch 3/15
586/586 [==============================] - 16s 27ms/step - loss: 0.6634 - accuracy: 0.6095
Epoch 4/15
586/586 [==============================] - 16s 27ms/step - loss: 0.6390 - accuracy: 0.6419
Epoch 5/15
586/586 [==============================] - 16s 27ms/step - loss: 0.6013 - accuracy: 0.6836
Epoch 6/15
586/586 [==============================] - 16s 27ms/step - loss: 0.5656 - accuracy: 0.7120
Epoch 7/15
586/586 [==============================] - 16s 27ms/step - loss: 0.5402 - accuracy: 0.7356
Epoch 8/15
586/586 [==============================] - 16s 27ms/step - loss: 0.5134 - accuracy: 0.7558
Epoch 9/15
586/586 [==============================] - 16s 27ms/step - loss: 0.4940 - accuracy: 0.7696
Epoch 10/15
586/586 [==============================] - 16s 27ms/step - loss: 0.472

# Transfer-Learning

In der Vorlesung wurde *Transfer-Learning* als effizientes Verfahren zum anpassen bereits vortrainierter neuronaler Netze beschrieben. Wenden die das Verfahren auch auf ihr Problem an. Verwenden sie dabei beispielsweise ein ResNet50 aus [Keras Applications](https://keras.io/api/applications/).




In [12]:
import tensorflow.keras.applications.resnet50 as resnet50
def preprocess(image):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = resnet50.preprocess_input(resized_image)
    return final_image

In [21]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from functools import partial

train_set=np.zeros((round((len(Xtr)/2)), 224,224,3))
for i in range(round(len(Xtr)/2)):
    train_set[i]=preprocess(Xtr[i])
print("trainSet resized")
test_set=np.zeros((len(Xte), 224,224,3))
for i in range(len(Xte)):
    test_set[i]=preprocess(Xte[i])
print("testSet resized")


trainSet resized
testSet resized


In [32]:
from tensorflow.keras.layers import GlobalAveragePooling2D

base_model=ResNet50(weights='imagenet', include_top=False)
avg=GlobalAveragePooling2D()(base_model.output)
output=Dense(1, activation="softmax")(avg)
model=Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(Xtr, ytr, epochs=5)

Epoch 1/5
586/586 [==============================] - 39s 56ms/step - loss: 0.6846 - accuracy: 0.4973
Epoch 2/5
586/586 [==============================] - 35s 60ms/step - loss: 0.6634 - accuracy: 0.4973
Epoch 3/5
586/586 [==============================] - 35s 60ms/step - loss: 0.6543 - accuracy: 0.4973
Epoch 4/5
586/586 [==============================] - 36s 61ms/step - loss: 0.6491 - accuracy: 0.4973
Epoch 5/5
586/586 [==============================] - 34s 59ms/step - loss: 0.6449 - accuracy: 0.4973


# Vergleich mit Feed-forward Netz

Vergleichen Sie ihre CNN-Ergebnisse mit den Ergebnisse aus dem Feed-fwd Netz